<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima_model import ARIMA
import warnings
warnings.filterwarnings("ignore")

C:\Users\212778050\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\212778050\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\212778050\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:

df = pd.read_csv('points.csv')
# print(df.head())
print(df.columns)

# get player values
idx = df.groupby(['id_player','web_name','singular_name_short'])['fixture'].transform(max) == df['fixture']
player_values = df[idx][['id_player','web_name','value','singular_name_short','first_name', 'second_name']]
player_values = player_values.set_index(['id_player','web_name','singular_name_short','first_name', 'second_name'])
player_values['arima'] = None



Index(['Unnamed: 0', 'id_player', 'web_name', 'singular_name_short',
       'first_name', 'second_name', 'element', 'fixture', 'opponent_team',
       'total_points', 'was_home', 'kickoff_time', 'team_h_score',
       'team_a_score', 'round', 'minutes', 'goals_scored', 'assists',
       'clean_sheets', 'goals_conceded', 'own_goals', 'penalties_saved',
       'penalties_missed', 'yellow_cards', 'red_cards', 'saves', 'bonus',
       'bps', 'influence', 'creativity', 'threat', 'ict_index', 'value',
       'transfers_balance', 'selected', 'transfers_in', 'transfers_out'],
      dtype='object')


In [3]:
    

for ix,row in player_values.iterrows():
    try:
        print(ix)
        player = ix[0]
        l = df[df['id_player']==player]['total_points']
        model = ARIMA(l, order=(2,1,0))
        model_fit = model.fit()
        output = model_fit.forecast(4)
        arima,output = np.mean(output[0]),output
        player_values.loc[ix,'arima'] = arima
        if len(l)>16:
            break
    except Exception as e:
        print(e)
        continue


(1, 'Leno', 'GKP', 'Bernd', 'Leno')


In [7]:
l
model = ARIMA(l, order=(2,1,0))
model_fit = model.fit()
output = model_fit.forecast(4)
arima,output = np.mean(output[0]),output
arima,output

(-0.15602466684466176,
 (array([-0.06648255, -0.15697081, -0.19866188, -0.20198342]),
  array([0.28828045, 0.48689078, 0.5193689 , 0.51956857]),
  array([[-0.63150185,  0.49853674],
         [-1.1112592 ,  0.79731757],
         [-1.21660623,  0.81928247],
         [-1.2203191 ,  0.81635227]])))

In [48]:
l = df[df['id_player']==69]['total_points']
model = ARIMA(l, order=(1,1,1))
model_fit = model.fit()
# output = model_fit.forecast(4)
# np.mean(output[0])

ZeroDivisionError: division by zero

In [41]:
player_values['arima']


id_player  web_name         singular_name_short  first_name  second_name        
1          Leno             GKP                  Bernd       Leno                     -0.208028
2          Rúnarsson        GKP                  Rúnar Alex  Rúnarsson                     None
532        Hein             GKP                  Karl        Hein                          None
559        Ramsdale         GKP                  Aaron       Ramsdale                   7.61532
572        Okonkwo          GKP                  Arthur      Okonkwo                       None
28         Steer            GKP                  Jed         Steer                 -1.33198e-08
30         Martínez         GKP                  Emiliano    Martínez                    2.6175
596        Sinisalo         GKP                  Viljami     Sinisalo                      None
80         Raya             GKP                  David       Raya Martin               -1.51817
92         Balcombe         GKP                  Ellery

In [3]:
# get per player stats
agg = df.groupby(['id_player','web_name','singular_name_short','first_name', 'second_name'], as_index=False).agg(
                      {'total_points':['mean','std','count']})
agg.columns = ['id_player','web_name','singular_name_short','first_name', 'second_name',
              'mean_points','std_points','count_points']

# get per position stats
group_stats = agg[agg['mean_points']>0].groupby(['singular_name_short'], as_index=False).agg(
                      {'mean_points':['mean','std','count']})
group_stats.columns = ['singular_name_short','group_mean','group_std','group_count']

# merge together position stats and player stats
agg = agg.merge(group_stats,
         how='inner',
         left_on='singular_name_short',
         right_on='singular_name_short')

agg['z_score'] = (agg['mean_points'] - agg['group_mean'])/agg['group_std']
agg['confidence'] = (agg['mean_points']* np.sqrt(agg['count_points']))/agg['std_points'] 
agg = agg.set_index(['id_player','web_name','singular_name_short','first_name', 'second_name'])


total_points = agg[[ 'mean_points', 'std_points', 'count_points',
#        'group_mean', 'group_std', 'group_count'
         'z_score', 'confidence']]


In [4]:
m = player_values.merge(total_points,
                   how='inner',
                   left_index=True,
                   right_index=True)
m['value'] = m['value'] / 10
m['per_dollar'] = m['mean_points'] / m['value']
m = m.sort_values('per_dollar', ascending=False).reset_index()
m.to_csv('player_and_values.csv', index=False)

In [5]:
def get_id(web_name,m):
    return m[(m['first_name']==web_name[0])&
            (m['second_name']==web_name[1])]['id_player'].values[0]

def get_value(id,m):
    return m[m['id_player']==id]['value'].values[0]

def get_pos(id,m):
    return m[m['id_player']==id]['singular_name_short'].values[0]


list_of_players = [
      ['Aaron','Ramsdale'],
      ['João Pedro Cavaco','Cancelo'],
      ['Ben' , 'White'],
      ['Tino', 'Livramento'],
      ['Raphael','Dias Belloli'],
      ['Conor' ,'Gallagher'],
      ['Emile','Smith Rowe'],
      ['Mohamed', 'Salah'],
      ['Emmanuel','Dennis'],
      ['Jamie','Vardy'],
      ['Cristiano Ronaldo', 'dos Santos Aveiro'],
      ['Ben' , 'Foster'],
      ['César','Azpilicueta'],
      ['Pontus','Jansson'],
    ['Allan','Marques Loureiro']
]

list_of_current_ids = []
for player in list_of_players:
    id = get_id(player,m)
    list_of_current_ids.append(id)
    
    
money_left_over = 0.9
number_of_transfers = 1

for player in list_of_players:
    print('----------------------------')
    print(player)
    id = get_id(player,m)
    current_val = get_value(id,m)
    pos = get_pos(id,m)
    total_val = money_left_over + current_val
    print(player)
    sub_table = m[((m['value']<=total_val)&
              (m['singular_name_short']==pos)&
                 (~m['id_player'].isin(list_of_current_ids)))].sort_values('per_dollar', ascending=False)[['id_player',
                                                                                           'web_name',
                                                                                           'value',
                                                                                           'mean_points',
                                                                                           'per_dollar',
                                                                                          'z_score',
                                                                                           'confidence']][:4]
    
    curr_player = m[m['id_player']==id][['id_player',
                           'web_name',
                           'value',
                           'mean_points',
                           'per_dollar',
                          'z_score',
                           'confidence']]
    sub_table = sub_table.append(curr_player)
    current_mean_points = m[m['id_player']==id]['mean_points'].values[0]
    sub_table['added_value'] = m['mean_points'] - current_mean_points
    print(sub_table)

----------------------------
['Aaron', 'Ramsdale']
['Aaron', 'Ramsdale']
    id_player  web_name  value  mean_points  per_dollar   z_score  confidence  \
4         475        Sá    5.2     4.571429    0.879121  1.707034    6.357755   
10        270    de Gea    5.2     4.090909    0.786713  1.399457    7.383714   
13        353    Lloris    5.4     4.200000    0.777778  1.469285    6.048927   
20         69   Sánchez    4.6     3.409091    0.741107  0.963029    6.299143   
0         559  Ramsdale    5.1     4.600000    0.901961  1.725322    6.118324   

    added_value  
4     -0.028571  
10    -0.509091  
13    -0.400000  
20    -1.190909  
0      0.000000  
----------------------------
['João Pedro Cavaco', 'Cancelo']
['João Pedro Cavaco', 'Cancelo']
   id_player  web_name  value  mean_points  per_dollar   z_score  confidence  \
1        678  Trippier    5.0     4.500000    0.900000  2.172001    1.800000   
2        429     Coady    4.7     4.190476    0.891591  1.935854    6.892692 

['Pontus', 'Jansson']
    id_player  web_name  value  mean_points  per_dollar   z_score  confidence  \
1         678  Trippier    5.0     4.500000    0.900000  2.172001    1.800000   
2         429     Coady    4.7     4.190476    0.891591  1.935854    6.892692   
3         676    Kamara    4.5     4.000000    0.888889  1.790532    1.589439   
5         439    Kilman    4.5     3.761905    0.835979  1.608880    7.226752   
53         76   Jansson    4.6     2.869565    0.623819  0.928081    5.158025   

    added_value  
1      1.630435  
2      1.320911  
3      1.130435  
5      0.892340  
53     0.000000  
----------------------------
['Allan', 'Marques Loureiro']
['Allan', 'Marques Loureiro']
     id_player   web_name  value  mean_points  per_dollar   z_score  \
36         426   Moutinho    4.9     3.238095    0.660836  1.139801   
47         365   Højbjerg    5.0     3.200000    0.640000  1.111447   
51         222  Henderson    5.1     3.227273    0.632799  1.131746   
58        

In [6]:
m['singular_name_short'].unique()

list_of_pos = ['DEF', 'MID', 'FWD', 'GKP']

for pos in list_of_pos:
    pos_df = m[m['singular_name_short']=='FWD']
    pos_df
pos_df.sort_values('arima', ascending=False)

,id_player,web_name,singular_name_short,first_name,second_name,value,arima,mean_points,std_points,count_points,z_score,confidence,per_dollar
22,450,Dennis,FWD,Emmanuel,Dennis,6.1,None,4.428571,4.717748,21,2.040566,4.301685,0.725995
50,586,Edouard,FWD,Odsonne,Edouard,6.4,None,4.052632,3.950720,19,1.756521,4.471340,0.633224
62,307,Saint-Maximin,FWD,Allan,Saint-Maximin,6.7,None,4.000000,3.114482,21,1.716755,5.885506,0.597015
78,518,Broja,FWD,Armando,Broja,5.2,None,2.909091,3.476576,22,0.892507,3.924794,0.559441
80,468,King,FWD,Joshua,King,5.8,None,3.238095,3.819748,21,1.141090,3.884763,0.558292
83,63,Maupay,FWD,Neal,Maupay,6.5,None,3.590909,2.986622,22,1.407662,5.639433,0.552448
87,413,Antonio,FWD,Michail,Antonio,7.9,None,4.347826,4.281129,23,1.979558,4.870548,0.550358
89,315,Pukki,FWD,Teemu,Pukki,5.8,None,3.181818,2.648204,22,1.098569,5.635535,0.548589
109,78,Toney,FWD,Ivan,Toney,6.6,None,3.347826,3.171015,23,1.223998,5.063240,0.507246
121,583,Hwang,FWD,Hee-Chan,Hwang,5.4,None,2.611111,3.483387,18,0.667366,3.180240,0.483539


In [41]:
df[df['id_player']==233]['total_points']

l = df[df['id_player']==233]['total_points'].append(df[df['id_player']==233]['total_points'])

def calc_arima(l):
    model = ARIMA(l, order=(2,1,1))
    model_fit = model.fit()
    output = model_fit.forecast(4)
    print(output)
#     return output[0].mean(),output

calc_arima(l)

(array([11.13400604, 10.49962824, 10.36168839, 10.41295923]), array([5.65466192, 5.70531748, 5.71228693, 5.71298549]), array([[ 0.05107233, 22.21693974],
       [-0.68258853, 21.68184502],
       [-0.83418826, 21.55756504],
       [-0.78428657, 21.61020503]]))
